In [1]:
!git clone https://github.com/css20090922/address_extraction.git

fatal: destination path 'address_extraction' already exists and is not an empty directory.


In [2]:
%cd address_extraction/

/content/address_extraction


In [3]:
!pip install transformers -U

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.4.2)


In [4]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
import pandas as pd
import csv
import matplotlib.pyplot as plt
from time import time
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig,TFBertForTokenClassification,create_optimizer
from sklearn.model_selection import train_test_split

In [5]:
MAX_LEN = 128
ADDR_MAX_LEN = 64
configuration = BertConfig()

In [6]:
MODEL_NAME = "cahya/bert-base-indonesian-522M"

slow_tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
save_path = "bert-base-indonesian-522M/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

tokenizer = BertTokenizer("bert-base-indonesian-522M/vocab.txt", lowercase=True,pad_token='0',mask_token='1')

In [7]:
df = pd.read_csv('/content/address_extraction/datasets/train.csv', encoding='latin-1')
df.set_index('id', inplace=True)

df['POI'] = df['POI/street'].str.split("/",n=2,expand=True)[0]
df['street'] = df['POI/street'].str.split("/",n=2,expand=True)[1]


df.head()

,raw_address,POI/street,POI,street
id,,,,
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,"aye, jati sampurna",/,,
2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,"toko dita, kertosono",toko dita/,toko dita,
4,jl. orde baru,/jl. orde baru,,jl. orde baru


記得tokenize前要先個別取出來

In [35]:
class preprocessor:
  def __init__(self, raw_address, POI, street):
    self.raw_address = raw_address
    self.attention_mask = np.zeros((1, MAX_LEN))
    self.skip = False

    self.label = [] # None: 0, street: 1,  POI: 2
    self.POI = str(POI).split()
    self.street = str(street).split()

  def preprocess(self):
    raw_address = self.raw_address
    POI = self.POI
    street = self.street
    # if len(POI) == 0 and len(street) == 0:
    if False:
      self.skip = True
    else:
      # Tokenize raw_address
      try:
        tokenized_raw_address = tokenizer(raw_address,padding='max_length',max_length=MAX_LEN,return_tensors="np")
        # tokenized_POI = tokenizer(POI,padding='max_length',max_length=ADDR_MAX_LEN,return_tensors="np",return_attention_mask=False)
        # tokenized_street = tokenizer(street,padding='max_length',max_length=ADDR_MAX_LEN,return_tensors="np",return_attention_mask=False)
        
        self.raw_address = tokenized_raw_address["input_ids"]
        self.attention_mask = tokenized_raw_address["attention_mask"]
        # self.POI = tokenized_POI["input_ids"]
        # self.street = tokenized_street["input_ids"]

        for word in raw_address.split(' '):
          # remove ',' from word
          word = word.replace(',', '')
          # print(word)
          if word in POI:
            self.label.append(2)
          elif word in street:
            self.label.append(1)
          else:
            self.label.append(0)
        self.label = self.label + [0]*(MAX_LEN - len(self.label))
        # print(self.label)
        
      except (TypeError,AssertionError):
        self.skip = True
        print("raw_address:"+str(raw_address))
        
def create_datasets(raw_data):
    datasets = []
 
    # split col poi and street
    for attr in raw_data:
      raw_address = attr[0]
      POI = attr[2]
      street = attr[3]
      
      squad_eg = preprocessor(raw_address, POI, street)
      squad_eg.preprocess()
      datasets.append(squad_eg)
    return datasets

def create_inputs_targets(token_data):
    dataset_dict = {
        "input_ids": [],
        "attention_mask": [],
        "label": []
    }
    for item in token_data:
        if item.skip == False:
            dataset_dict["input_ids"].append(item.raw_address[0])
            dataset_dict["attention_mask"].append(item.attention_mask[0])               
            # dataset_dict["POI"].append(item.POI[0])
            # dataset_dict["street"].append(item.street[0])
            dataset_dict['label'].append(item.label)

    x = [
        dataset_dict["input_ids"],
        dataset_dict["attention_mask"]
    ]
    y = dataset_dict['label']
    return x, y

In [36]:
start = time()
# token_df = create_datasets(df[:10].to_numpy().tolist())
token_df = create_datasets(df.to_numpy().tolist())
x_train, y_train = create_inputs_targets(token_df)
x_train = np.array(x_train)
y_train = np.array(y_train)
print("cost {}".format(time()-start))

cost 107.30913662910461


In [37]:
x_train.shape, y_train.shape

((2, 300000, 128), (300000, 128))

In [38]:
def create_model():
  encoder = TFBertModel.from_pretrained("cahya/bert-base-indonesian-522M")

  input_ids = layers.Input(shape=(MAX_LEN,), dtype=tf.int32)
  attention_mask = layers.Input(shape=(MAX_LEN,), dtype=tf.int32)
  embedding = encoder(
      input_ids,attention_mask=attention_mask
  )[0]

  # output_POI = layers.Dense(ADDR_MAX_LEN, name="output_POI", use_bias=False)(embedding)
  # output_POI = layers.Flatten()(output_POI)

  # output_street = layers.Dense(ADDR_MAX_LEN, name="output_street", use_bias=False)(embedding)
  # output_street = layers.Flatten()(output_street)

  # output_POI = layers.Activation(keras.activations.softmax)(output_POI)
  # output_street = layers.Activation(keras.activations.softmax)(output_street)
  dense = layers.Dense(3)(embedding)
  # output = layers.Flatten()(dense)

  model = keras.Model(
      inputs=[input_ids,attention_mask],
      # outputs=output
      outputs=dense
  )

  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  optimizer = keras.optimizers.Adam(lr=5e-5)
  model.compile(optimizer=optimizer, loss=[loss, loss])
  return model

In [39]:
model = create_model()
model.summary()

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_2 (TFBertModel)   TFBaseModelOutputWit 110617344   input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128, 3)       2307        tf_bert_model_2[0][0]      

In [ ]:
start=time()
history = model.fit(
    [x_train[0], x_train[1]],
    y_train,
    epochs=3,  # For demonstration, 3 epochs are recommended
    verbose=1,
    batch_size=64,
    # steps_per_epoch = 100,
    validation_split=0.2
)
print("cost"+str(time()-start))
print(history)

Epoch 1/3
 397/3750 [==>...........................] - ETA: 1:21:45 - loss: 0.3317

In [26]:
plt.plot(trainacc)
plt.plot(valacc)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

NameError: ignored